# Get data

In [ ]:
from awpy import Demo

# folder of demos
path = ".demos/"

# match one between Faze and Cloud 9 in the Perfect World Shanghai RMR for 4th place
match = "faze-vs-cloud-9-m1-mirage.dem"

# load demo
dem = Demo(path + match)

In [ ]:
# other tables from demoparser that could be useful
for event_name, event in dem.events.items():
    print(f"{event_name}: {event.shape[0]} rows x {event.shape[1]} columns")

item_equip: 9710 rows x 44 columns
round_freeze_end: 20 rows x 16 columns
weapon_reload: 96 rows x 36 columns
player_blind: 216 rows x 58 columns
chat_message: 8 rows x 37 columns
bomb_planted: 10 rows x 38 columns
round_poststart: 19 rows x 16 columns
flashbang_detonate: 110 rows x 40 columns
player_death: 140 rows x 95 columns
bomb_begindefuse: 6 rows x 37 columns
inferno_startburn: 80 rows x 40 columns
hltv_fixed: 111 rows x 24 columns
bomb_exploded: 2 rows x 38 columns
round_announce_last_round_half: 1 rows x 16 columns
announce_phase_end: 1 rows x 16 columns
item_pickup: 1256 rows x 39 columns
player_team: 10 rows x 41 columns
round_officially_ended: 36 rows x 16 columns
decoy_detonate: 2 rows x 40 columns
cs_pre_restart: 19 rows x 16 columns
round_announce_match_point: 1 rows x 16 columns
hltv_chase: 395 rows x 23 columns
cs_win_panel_match: 1 rows x 16 columns
bomb_dropped: 48 rows x 37 columns
bomb_defused: 3 rows x 38 columns
bomb_beginplant: 11 rows x 37 columns
hltv_versioni

# Rounds and Outcomes

* total number of rounds played
* win rate of each team (CT and T)
* Rounds where the bomb was planted but not defused

## total rounds

In [170]:
# one row per round so the number of rows is the number of rounds
dem.rounds.shape[0]

19

There were 19 rounds played

## win rate of each team

In [ ]:
# labelling which team was CT and T in the first / second half

# obtaining the first team name and which side it was on and assigning values based on that
if list(dem.kills[['victim_team_name', 'victim_team_clan_name']].iloc[0, :])[0] == "CT":

    # first value is CT in this match
    first_half_CT = list(dem.kills[['victim_team_name', 'victim_team_clan_name']].iloc[0, :])[1]

    # assign T based on CT
    first_half_T = dem.kills.loc[dem.kills['victim_team_clan_name'] != first_half_CT]['victim_team_clan_name'].iloc[0,]

    # switch for second half
    second_half_CT, second_half_T = first_half_T, first_half_CT

else:
    # everything should just be flipped in the alternative case

    first_half_T = list(dem.kills[['victim_team_name', 'victim_team_clan_name']].iloc[0, :])[1]

    # assign T based on CT
    first_half_CT = dem.kills.loc[dem.kills['victim_team_clan_name'] != first_half_T]['victim_team_clan_name'].iloc[0,]

    # switch for second half
    second_half_CT, second_half_T = first_half_T, first_half_CT
    

print(f"CT 1st half: {first_half_CT}\n T 1st half: {first_half_T}\nCT 2nd half: {second_half_CT}\n T 2nd half: {second_half_T}")

CT 1st half: FaZe Clan
 T 1st half: Cloud 9
CT 2nd half: Cloud 9
 T 2nd half: FaZe Clan


In [ ]:
import pandas as pd
import numpy as np

# assigning team name to round based on half and which in game team won
dem.rounds['winning_team'] = np.select(
    [
        (dem.rounds['round'].between(1, 12, inclusive='both')) & (dem.rounds['winner'] == "CT"), 
        (dem.rounds['round'].between(1, 12, inclusive='both')) & (dem.rounds['winner'] == "T"),
        (dem.rounds['round'].between(13, 24, inclusive='both')) & (dem.rounds['winner'] == "CT"),
        (dem.rounds['round'].between(13, 24, inclusive='both')) & (dem.rounds['winner'] == "T")
    ], 
    [
        first_half_CT, 
        first_half_T,
        second_half_CT,
        second_half_T
        
    ], 
    default='Unknown'
)

# labelling halves
dem.rounds['half'] = np.select(
    [
        (dem.rounds['round'].between(1, 12, inclusive='both')) , 
        (dem.rounds['round'].between(13, 24, inclusive='both')) ,
    ], 
    [
        "First", 
        "Second",
        
    ], 
    default='Overtime'
)


In [ ]:
# counting number of round wins each team won per half
summary_series = dem.rounds.groupby(['winning_team', 'winner', 'half'])['round'].count()

# converting to table and sorting into coherent order
sum_table = pd.DataFrame(summary_series).sort_values(['winning_team', 'half'])


# finding win proportion per half
sum_table['proportion'] = np.select(
    [
        sum_table.index.get_level_values('half') == "First",
        sum_table.index.get_level_values('half') == "Second"
    ],
    [
        round(sum_table['round']/12, 4), # would ideal not like this not be hard coded
        round(sum_table['round']/7, 4)
    ],
    default='err'
)

sum_table

round proportion
winning_team winner half                    
Cloud 9      T      First       4     0.3333
             CT     Second      2     0.2857
FaZe Clan    CT     First       8     0.6667
             T      Second      5     0.7143

## Rounds where bomb was planted and not defused

In [196]:
# joining rounds with what the bomb did each round
rounds_bomb_table = dem.rounds.set_index('round').join(dem.bomb.set_index('round'))

# filtering to rounds where the bomb was actually planted
planted_not_defused_rounds = rounds_bomb_table[(rounds_bomb_table['event'] == 'planted')]

# filtering to rounds where bomb was not defused
planted_not_defused_rounds[planted_not_defused_rounds['reason'] != 'bomb_defused'][['winner', 'reason', 'winning_team', 'half', 'site', 'clock']]

,winner,reason,winning_team,half,site,clock
round,,,,,,
4,T,ct_killed,Cloud 9,First,BombsiteA,00:40
9,T,ct_killed,Cloud 9,First,BombsiteA,00:40
10,T,bomb_exploded,Cloud 9,First,BombsiteB,00:40
12,T,ct_killed,Cloud 9,First,BombsiteB,00:40
15,T,ct_killed,FaZe Clan,Second,BombsiteB,00:40
16,T,ct_killed,FaZe Clan,Second,BombsiteB,00:40
18,T,ct_killed,FaZe Clan,Second,BombsiteB,00:40


### Investigating round 10

Why did the bomb explode in round 10?

In [195]:
# number of kills in round 10
dem.kills[dem.kills['round'] == 10][['attacker_name', 'dmg_health', 'dmg_armor', 'attacker_team_name', 'victim_team_name', 'victim_name', 'victim_health', 'victim_armor_value', 'clock', 'ticks_since_bomb_plant']]

,attacker_name,dmg_health,dmg_armor,attacker_team_name,victim_team_name,victim_name,victim_health,victim_armor_value,clock,ticks_since_bomb_plant
66,Ax1Le,27,3,TERRORIST,CT,frozen,15,100,01:38,<NA>
67,rain,175,6,CT,TERRORIST,interz,100,87,00:51,<NA>
68,Ax1Le,27,3,TERRORIST,CT,broky,27,92,00:43,<NA>


Only 3 people were killed total in the round? 2 counter-terrorists and 1 terrorist

#### GIF of round 10

In [10]:
from awpy.plot import gif, PLOT_SETTINGS
from tqdm import tqdm
import os

if not os.path.isfile("de_mirage.gif"):
    frames = []

    for tick in tqdm(dem.ticks[dem.ticks["round"] == 10].tick.values[::128]):
        frame_df = dem.ticks[dem.ticks["tick"] == tick]
        frame_df = frame_df[
            ["X", "Y", "Z", "health", "armor_value", "pitch", "yaw", "team_name", "name"]
        ]

        points = []
        point_settings = []

        for _, row in frame_df.iterrows():
            points.append((row["X"], row["Y"], row["Z"]))

            # Determine team and corresponding settings
            team = "ct" if row["team_name"] == "CT" else "t"
            settings = PLOT_SETTINGS[team].copy()

            # Add additional settings
            settings.update(
                {
                    "hp": row["health"],
                    "armor": row["armor_value"],
                    "direction": (row["pitch"], row["yaw"]),
                    "label": row["name"],
                }
            )

            point_settings.append(settings)

        frames.append({"points": points, "point_settings": point_settings})

    print("Finished processing frames. Creating gif...")
    gif(f"{dem.header['map_name']}", frames, f"{dem.header['map_name']}.gif", duration=100)

Cloud9 (terrorists) got an early kill (Ax1Le -> frozen) in B-site and then fake rotated out. Faze over reacted and rotated to A and mid to control space. Faze was able to kill (rain -> interz) the lurking player outside of A-site and Cloud9 exploded into B-site. The player (broky) in B-site was not able to pick up any kills on Cloud9's entry and so the call seemingly was made to not retake.


Further questions:
* Were the CTs in a save round? It seems like Faze had some early aggression in Apts that did not play out for them?
* Did the CTs have defuse kits?
* How much health did the CTs have?

In [234]:
# filter to round 10, after the bomb was planted, alive players (non empty inventory), and CT players
ct_round_10_data = dem.ticks[(dem.ticks['round'] == 10) & (dem.ticks['is_bomb_planted'] == True) & (dem.ticks['inventory']) * (dem.ticks['team_name'] == "CT")]

In [286]:
# selection of relevant columns
ct_round_10_data.drop_duplicates(subset = "name", keep="first")[['name', 'inventory', 'health', 'armor_value', 'has_defuser', 'has_helmet', 'current_equip_value']]

,name,inventory,health,armor_value,has_defuser,has_helmet,current_equip_value
830962,rain,"[knife_butterfly, Desert Eagle, AK-47, Molotov]",68,96,True,True,5200
830966,ropz,"[knife_karambit, Five-SeveN]",100,100,False,False,1450
830968,karrigan,"[knife_karambit, USP-S, MP9]",75,84,False,False,2400


Very minimal equipment from the CTs: rain had a kit, helmet, but was hit for nearly 30 hp; ropz was at full health but had no helmet nor kit and only a pistol; karrigan had an smg, was already hit, had no kit nor helmet. 

Seems like a good call to not try to retake

In [ ]:
# the CTs did not pick up any new weapons after the round ended
dem.events['item_pickup'][(dem.events['item_pickup']['round'] == 10) & (dem.events['item_pickup']['is_bomb_planted'] == True)]

,ct_team_clan_name,ct_team_name,defindex,game_phase,game_time,is_bomb_planted,is_ct_timeout,is_freeze_period,is_match_started,is_technical_timeout,...,user_last_place_name,user_name,user_ping,user_pitch,user_steamid,user_team_clan_name,user_team_name,user_yaw,user_zoom_lvl,round


State of economy beginning of round 10 and round 11

In [292]:
# grab first tick of rounds 10 and 11 to see how much money the CTs have
ct_economy_rounds_10_11 = dem.ticks[(dem.ticks['round'].isin([10, 11])) & (dem.ticks['team_name'] == "CT")].drop_duplicates(subset = ['name', 'round'], keep = 'first')[['name', 'inventory', 'round', 'current_equip_value', 'armor_value', 'has_helmet']]

ct_economy_rounds_10_11

,name,inventory,round,current_equip_value,armor_value,has_helmet
779190,broky,[knife_butterfly],10,1500,100,True
779192,rain,"[knife_butterfly, Desert Eagle, Smoke Grenade,...",10,2600,100,True
779196,ropz,"[knife_karambit, Smoke Grenade]",10,1450,100,False
779197,frozen,"[knife_stiletto, USP-S]",10,2100,100,False
779198,karrigan,"[knife_karambit, USP-S, Smoke Grenade]",10,2400,100,False
874480,broky,"[knife_butterfly, USP-S]",11,200,0,False
874482,rain,"[knife_butterfly, AK-47, Molotov]",11,4700,96,True
874486,ropz,"[knife_karambit, Flashbang]",11,1350,26,False
874487,frozen,"[knife_stiletto, Desert Eagle]",11,700,0,False
874488,karrigan,"[knife_karambit, USP-S]",11,2100,84,False


In [296]:
ct_economy_rounds_10_11.groupby('round')['current_equip_value'].sum()

round
10    10050
11     9050
Name: current_equip_value, dtype: uint32

Look up HLTV common monetary boundaries but fairly sure these are semi-buy to nearly eco rounds

# Round Durations

## What is the average duration of the rounds?

In [333]:
rounds = dem.rounds.copy()

# calculate total number of ticks
rounds['total_ticks'] = rounds['end'] - rounds['start']

# fairly sure matches are played on 64 tick servers so divide ticks by 64 to get seconds
rounds['seconds'] = round(rounds['total_ticks']/64, 2)

# quartile ranges for number of seconds
qs = list(rounds['seconds'].quantile(q = [0, .25, .5, .75, 1]))

# add average time
qs.append(rounds['seconds'].mean())

# create table 
seconds_five_num_summary = pd.DataFrame(
    qs
).T.rename(mapper = {0: "Min", 1: "1st quartile", 2: "Median", 3: "3rd quartile", 4: "Max", 5: "Average"}, axis = 1)

seconds_five_num_summary

,Min,1st quartile,Median,3rd quartile,Max,Average
0,51.0,105.14,125.72,137.045,186.84,121.85


50% of rounds lasted just over 2 minutes and 5 seconds. The average round lasted just over 2 minutes.

## How many players achieved multi-kills?

In [334]:
# obtain player round team name information
player_info = dem.kills[['round', 'attacker_name', 'attacker_team_clan_name','attacker_team_name']].copy().drop_duplicates().set_index(['attacker_name', 'round'])

player_info

,,attacker_team_clan_name,attacker_team_name
attacker_name,round,,
Boombl4,1,Cloud 9,TERRORIST
ropz,1,FaZe Clan,CT
frozen,1,FaZe Clan,CT
Ax1Le,1,Cloud 9,TERRORIST
HeavyGod,1,Cloud 9,TERRORIST
...,...,...,...
Boombl4,18,Cloud 9,CT
rain,19,FaZe Clan,TERRORIST
karrigan,19,FaZe Clan,TERRORIST


In [383]:
# index is just player name and only column is which team they are on
player_teams = player_info.reset_index().drop_duplicates('attacker_name').set_index('attacker_name')['attacker_team_clan_name']

In [ ]:
kills = dem.kills.copy()

# number of multi-kills to look at
num_kills = 2

# count the number of rows each player gets in each round
# fill in missing values with 0 kills
# pivot table to make 1 row per round player to allow for filtering based on number of kills
kills_per_round = kills.groupby(['round', 'attacker_name'])['tick'].count().unstack().fillna(0).reset_index().melt(id_vars = "round", value_name = "kills")

# sort and set index to allow joining for more context of players
multikill_round_players = kills_per_round.sort_values('round').set_index(['attacker_name', 'round'])

# join information about player context and filter to number of multi-kills
multikills = multikill_round_players.join(player_info)[multikill_round_players['kills'] >= num_kills].reset_index()

# show unique players overall
multikills.drop_duplicates(subset = "attacker_name").reset_index(drop = True)

,attacker_name,round,kills,attacker_team_clan_name,attacker_team_name
0,broky,1,2.0,FaZe Clan,CT
1,Boombl4,1,2.0,Cloud 9,TERRORIST
2,ropz,1,2.0,FaZe Clan,CT
3,rain,2,2.0,FaZe Clan,CT
4,karrigan,3,2.0,FaZe Clan,CT
5,Ax1Le,4,3.0,Cloud 9,TERRORIST
6,frozen,8,2.0,FaZe Clan,CT
7,ICY,8,2.0,Cloud 9,TERRORIST
8,interz,9,2.0,Cloud 9,TERRORIST
9,HeavyGod,13,2.0,Cloud 9,CT


Every player had at least 1 round where they got a multi-kll (CT or T)

In [ ]:
# unique players by side
multikills.drop_duplicates(subset = ["attacker_name", 'attacker_team_name']).reset_index(drop = True).sort_values(['attacker_name', 'attacker_team_name'], ascending = [True, False])

,attacker_name,round,kills,attacker_team_clan_name,attacker_team_name
15,karrigan,19,2.0,FaZe Clan,TERRORIST
4,karrigan,3,2.0,FaZe Clan,CT
5,Ax1Le,4,3.0,Cloud 9,TERRORIST
1,Boombl4,1,2.0,Cloud 9,TERRORIST
9,HeavyGod,13,2.0,Cloud 9,CT
7,ICY,8,2.0,Cloud 9,TERRORIST
14,broky,17,2.0,FaZe Clan,TERRORIST
0,broky,1,2.0,FaZe Clan,CT
10,frozen,13,2.0,FaZe Clan,TERRORIST
6,frozen,8,2.0,FaZe Clan,CT


All FaZe members got a multi-kill on both halves. On Cloud 9: Ax1Le, Boombl4, and Icy did not get multi-kills on CT; HeavyGod did not get a multi-kill on T; and interz got a multi-kill on both halves.

#### Who got the most multi-kills?

In [394]:
# group by player name and count the number of rows (rounds with multi-kills) then sort by descending to find most
total_multikills = multikills.groupby(['attacker_name'])['attacker_team_clan_name'].count().reset_index().sort_values('attacker_team_clan_name', ascending=False).rename(mapper = {'attacker_team_clan_name': "number_multi_kills"}, axis = 1).set_index('attacker_name')

total_multikills.join(player_teams)

,number_multi_kills,attacker_team_clan_name
attacker_name,,
rain,7,FaZe Clan
ropz,6,FaZe Clan
HeavyGod,4,Cloud 9
frozen,4,FaZe Clan
karrigan,3,FaZe Clan
Ax1Le,3,Cloud 9
Boombl4,2,Cloud 9
broky,2,FaZe Clan
interz,2,Cloud 9


rain got the most multi-kills

I wonder if getting a multi-kill in a round is a good indicator if you will win that round.

Conversely, how many rounds did a team *lose* after getting a mutli-kill in a round?